In [1]:
"""
    Notebook name: LIFESPAN capacity and feed
    Usage: This notebook belongs to LIFESPAN and it is used to create two maps,
           one of Landfills clasification and another one of Disposed tones to landfill.
    DF used: 'data/DF_TOTAL.csv'
    Output: graphs/Landfills clasification.html
            graphs/Disposed tons to landfill.html
"""

"\n    Notebook name: LIFESPAN capacity and feed\n    Usage: This notebook belongs to LIFESPAN and it is used to create two maps,\n           one of Landfills clasification and another one of Disposed tones to landfill.\n    DF used: 'data/DF_TOTAL.csv'\n    Output: graphs/Landfills clasification.html\n            graphs/Disposed tons to landfill.html\n"

## Packages & Databases

In [2]:
# Imports the necessary libraries
import pandas              as pd 
import matplotlib.pyplot   as plt 
import seaborn             as sns; sns.set()
import branca
import geopandas
import requests
from colour import Color
from folium.plugins import MarkerCluster

import folium # package for making maps, please make sure to use a version older than 1.0.0.

from IPython.display import display
from folium.plugins import TimeSliderChoropleth
from folium.plugins import HeatMap

# from time_slider_choropleth import TimeSliderChoropleth
%matplotlib inline

# Imports lifespan dataset
DF_TOTAL=pd.read_csv("data/DF_TOTAL.csv")

## Databases

In [3]:
# IRRELEVANT DATA
DF_TOTAL.drop(["Unnamed: 0"],axis=1,inplace=True)
# LOCATION
DF_TOTAL['AUX_LOC'] = DF_TOTAL.apply(lambda x:'%s-%s' % (x['Nombre del departamento'],x['Nombre del municipio']),axis=1)

my_zip = zip(DF_TOTAL['Latitud'], DF_TOTAL['Longitud'], DF_TOTAL['LIFESPAN_PERCENTAGE'])
list_of_my_zip = list(my_zip)

DF_TOTAL["code"]=DF_TOTAL["Categories"].astype("category").cat.codes
DF_TOTAL

,Id Empresa,Empresa,Nombre del departamento,Nombre del municipio,Nombre sitio disposición final,NUSD,Fecha inicio operaciones,Tipo disposición final,Longitud,Latitud,...,Fecha Res. permiso vertimiento,Fecha term permiso vertimiento,Fecha aplicación metodología,Fecha de certificación,TONELADAS_DISPUESTAS,LIFESPAN,LIFESPAN_PERCENTAGE,Categories,AUX_LOC,code
0,21413,NUEVO MONDOÑEDO S.A. E.S.P.,CUNDINAMARCA,BOJACA,RELLENO SANITARIO NUEVO MONDOÑEDO,1137525099,2007-01-15 00:00:00,Relleno sanitario,-74.285833,4.645556,...,2008-11-24 00:00:00,2029-12-31 00:00:00,2016-04-30 00:00:00,25/01/2019,811233.50,6290766.50,88.577394,GOOD,CUNDINAMARCA-BOJACA,1
1,2147,BIORGANICOS DEL SUR DEL HUILA S.A E.S.P.,HUILA,PITALITO,Planta de tramiento de Residuos Solidos,750441551,2003-03-06 00:00:00,Relleno sanitario,-76.123177,1.872645,...,2010-12-20 00:00:00,2019-05-18 00:00:00,2016-07-13 00:00:00,04/12/2019,68698.21,179176.60,72.285118,GOOD,HUILA-PITALITO,1
2,22432,EMPRESAS PUBLICAS DE YAGUARA S.A. E.S.P.,HUILA,YAGUARA,RELLENO SANITARIO LOS ANGELES,368141885,2006-12-26 00:00:00,Relleno sanitario,-75.135422,2.594600,...,2006-12-26 00:00:00,2016-12-25 00:00:00,2016-01-01 00:00:00,13/12/2016,210493.88,2943360.12,93.325820,GOOD,HUILA-YAGUARA,1
3,1874,AMBIENTAR E.S.P. S.A.,GUAVIARE,SAN JOSE DEL GUAVIARE,EL ALGARROBO,960695001,1996-08-19 00:00:00,celda de contingencia,-72.360026,-2.322336,...,2015-05-22 00:00:00,2017-05-22 00:00:00,2016-04-01 00:00:00,28/06/2018,28052.88,285851.02,91.063227,GOOD,GUAVIARE-SAN JOSE DEL GUAVIARE,1
4,4933,UNIDAD DE SERVICIOS PUBLICOS DOMICILIARIOS DEL...,GUAVIARE,CALAMAR,RELLENO SANITARIO EL DIAMANTE,716595015,2010-10-04 00:00:00,celda transitoria,-67.000000,13.020345,...,2009-05-19 00:00:00,2040-05-18 00:00:00,2040-05-18 00:00:00,26/09/2017,927.10,48925.90,98.140333,GOOD,GUAVIARE-CALAMAR,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,32194,EMPRESA DE SERVICIOS PUBLICOS DEL ALTO BAUDO E...,CHOCO,ALTO BAUDO,CAMPO NUEVO,2275427025,2016-01-01 00:00:00,Relleno sanitario,-76.581993,5.311044,...,NaN,NaN,2016-01-01 00:00:00,08/07/2021,130.58,1369.42,91.294667,GOOD,CHOCO-ALTO BAUDO,1
139,36394,ECOAMBIENTAL ACTIVA DE COLOMBIA S.A. ESP,ANTIOQUIA,CAUCASIA,PARQUE AMBIENTAL CAMPOALEGRE,1101605154,2018-01-15 00:00:00,Relleno sanitario,-75.142725,8.004989,...,NaN,NaN,2018-01-15 00:00:00,13/12/2018,63394.97,478034.03,88.291176,GOOD,ANTIOQUIA-CAUCASIA,1
140,39056,INTERASEO DEL ARCHIPIELAGO S.A.S. E.S.P.,"ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANT...",SAN ANDRES,MAGIC GARDEN,966388001,1985-04-01 00:00:00,Relleno sanitario,-81.726689,12.554538,...,NaN,NaN,2017-10-17 00:00:00,04/07/2018,45369.66,554630.34,92.438390,GOOD,"ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANT...",1
141,44197,UNIÓN TEMPORAL OPERACIÓN RELLENO SANITARIO REG...,LA GUAJIRA,MAICAO,UNION TEMPORAL OPERACION RELLENO SANITARIO REG...,1678844430,2019-04-01 00:00:00,Relleno sanitario,-72.366944,11.402222,...,NaN,NaN,2019-07-22 00:00:00,08/09/2020,15517.82,2092149.18,99.263744,GOOD,LA GUAJIRA-MAICAO,1


In [4]:
# Map settings
# color scheme
colors1=["red","green","yellow"]

ny_coords = [4.624335, -74.063644] # lat, long
mapa_kg_per_capita = folium.Map(location=ny_coords, zoom_start=5.25, tiles="cartodbpositron")


style_function = lambda x: {
    'fillColor': colormap(x['properties']['KG_promedio_dispuestos_per_capita']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.8
}


for indx, row in DF_TOTAL.iterrows():
    lat = row['Latitud']
    long = row['Longitud']
    toneladas_disp_map = row['TONELADAS_DISPUESTAS']
    code_color=row['code']
    Categories_map = row['Categories']
    Lifespan=row['LIFESPAN_PERCENTAGE']
    Municipio=row['Nombre del municipio']
    Province=row["Nombre del departamento"]
    Relleno=row['Nombre sitio disposición final']
    
    color = colors1[code_color]
    
    folium.Circle(
        [lat, long],
        radius=1,
        tooltip=f'''<i>Province: {Province}<br>
        <i>Municipality: {Municipio}<br>
        <i>Landfill name: {Relleno}<br>
        <i>Waste disposal: {toneladas_disp_map:.2f} Tons<br>
        <i>Landfill capacity: {Lifespan:.2f}%<br>
        <i>Category: {Categories_map}''',
        weight=toneladas_disp_map/40000,
        color=color).add_to(mapa_kg_per_capita)
        
    
    # Stop at 32 points 
    if indx == 10000:
        break

folium.LayerControl().add_to(mapa_kg_per_capita)
# Saves the map on html format
mapa_kg_per_capita.save("graphs/Disposed tons to landfill.html")
mapa_kg_per_capita

In [5]:

ny_coords = [4.624335, -74.063644] # lat, long
mapa_kg_per_capita = folium.Map(location=ny_coords, zoom_start=5.25, tiles="cartodbpositron")


for indx, row in DF_TOTAL.iterrows():
    lat = row['Latitud']
    long = row['Longitud']
    toneladas_disp_map = row['TONELADAS_DISPUESTAS']
    code_color=row['code']
    Categories_map = row['Categories']
    Lifespan=row['LIFESPAN_PERCENTAGE']
    Municipio=row['Nombre del municipio']
    Relleno=row['Nombre sitio disposición final']
    Province=row['Nombre del departamento']
    
    color = colors1[code_color]
    
    folium.Circle(
        [lat, long],
        radius=1,
        tooltip=f'''<i>Province: {Province}<br>
        <i>Municipality: {Municipio}<br>
        <i>Landfill name: {Relleno}<br>
        <i>Waste disposal: {toneladas_disp_map:.2f} Tons<br>
        <i>Landfill capacity: {Lifespan:.2f}%<br>
        <i>Category: {Categories_map}''',
        weight=Lifespan/8,
        color=color,name="kevin").add_to(mapa_kg_per_capita)
        
    
    # Stop at 32 points 
    if indx == 10000:
        break
        
# Saves the map on html format
mapa_kg_per_capita.save("graphs/Landfills clasification.html")
mapa_kg_per_capita